In [1]:
import rds_config
from sqlalchemy import create_engine

#rds settings
db_host  = rds_config.db_host
db_username = rds_config.db_username
db_password = rds_config.db_password
db_name = rds_config.db_name

#한글 깨짐 현상 해결
#http://yongho1037.tistory.com/570
SQLALCHEMY_DATABASE_URI = 'mysql+pymysql://' + db_username + ':' + db_password + '@' + db_host + '/' + db_name + '?charset=utf8'

engine = create_engine(SQLALCHEMY_DATABASE_URI)
connection = engine.connect()

In [2]:
import pandas as pd
sql = "SELECT date, substring(date, 6,2) as month, media, title, org_count, naver_count, org_count+naver_count as total_count, section FROM newscrawl WHERE org_count>0 or naver_count>0 "
df = pd.read_sql(sql, connection)
#pd.read_csv("dcdata10.csv")

In [3]:
len(df)

450032

In [4]:
df.tail(10)

,date,month,media,title,org_count,naver_count,total_count,section
450022,2016-12-14,12,jtbc,"박 대통령 '공짜 진료' 논란…특검, 뇌물죄 적용 고심",2,12,14,사회
450023,2016-12-14,12,jtbc,"[오늘] 12월 14일…'최순실 육성' 공개, 분명한 메시지",4,34,38,정치
450024,2016-12-14,12,jtbc,[앵커브리핑] 눈먼 자들의 국가…감출 수 없는 진실들,6902,53,6955,정치
450025,2016-12-14,12,jtbc,다시 한 번 또 봐도…'무능했던' 청와대의 7시간 대응,68,23,91,정치
450026,2016-12-14,12,jtbc,"최순실 ""조작으로 몰아라""…태블릿PC 소유 자백한 셈",166,30,196,사회
450027,2016-12-14,12,jtbc,[밀착카메라] 청문회 준비 어떻게…국조 위원들 24시,14,37,51,정치
450028,2016-12-14,12,jtbc,다급했던 최순실 통화…JTBC 보도 사흘 뒤 '은폐 지시',9175,50,9225,사회
450029,2016-12-14,12,jtbc,"[비하인드 뉴스] 대통령 미용사, 버킹엄궁도 따라갔다",4,0,4,정치
450030,2016-12-14,12,jtbc,"[팩트체크] 개헌, 정말로 '두 달 반' 안에 가능할까?",1119,30,1149,정치
450031,2016-12-14,12,jtbc,"[Talk쏘는 정치] ""걸핏하면 시위해"" 촛불 폄하한 교수",6,15,21,정치


In [5]:
%%time
from konlpy.tag import Mecab
mecab = Mecab()

def mecab_words(sentence):
    word_list = mecab.nouns(sentence)
    #return " ".join(word_list)
    return " ".join(filter(lambda x: len(x)>1, word_list))
    #return filter(lambda x: len(x)>1, " ".join(word_list))

df['title_me_noun'] = df['title'].apply(mecab_words)

CPU times: user 32.5 s, sys: 128 ms, total: 32.6 s
Wall time: 33.7 s


In [7]:
df_by_month = df.groupby(['month'])
df_by_media_month = df.groupby(['month', 'media'])

In [9]:
from collections import Counter

by_me_month = df_by_month['title_me_noun'].apply(list)
result_by_me_month = pd.Series()

for month, words in by_me_month.iteritems():
    result_by_me_month.set_value(month, Counter(" ".join(words).split(" ")))

In [10]:
for month, words in result_by_me_month.iteritems():
    print(month, words.most_common(30))

01 [('종합', 1493), ('한국', 1147), ('대통령', 913), ('중국', 845), ('민주', 804), ('경제', 786), ('뉴스', 761), ('서울', 692), ('위안부', 681), ('정부', 660), ('세계', 616), ('북한', 610), ('시장', 602), ('국민', 578), ('논란', 570), ('한파', 549), ('누리', 529), ('안철수', 521), ('미국', 511), ('영상', 497), ('카드', 494), ('정치', 468), ('의원', 452), ('일본', 448), ('청년', 444), ('가능', 439), ('기업', 421), ('핵실험', 408), ('공항', 405), ('단독', 397)]
02 [('종합', 1252), ('한국', 916), ('개성', 882), ('공단', 879), ('대통령', 737), ('뉴스', 689), ('중국', 688), ('정부', 670), ('사드', 666), ('북한', 662), ('국민', 659), ('민주', 650), ('미사일', 649), ('테러', 583), ('서울', 560), ('기업', 552), ('경제', 551), ('세계', 541), ('시장', 533), ('영상', 494), ('제재', 492), ('가능', 489), ('버스터', 478), ('필리', 478), ('논란', 456), ('카드', 452), ('발사', 448), ('의원', 448), ('총선', 425), ('대북', 422)]
03 [('공천', 1341), ('종합', 1151), ('민주', 1070), ('한국', 994), ('총선', 881), ('국민', 874), ('김종인', 820), ('뉴스', 819), ('후보', 766), ('대통령', 756), ('대표', 686), ('서울', 685), ('이세돌', 606), ('테러', 598), ('유승민', 58

In [11]:
df_by_month.agg(['mean', 'count'])

org_count        naver_count        total_count       
             mean  count        mean  count        mean  count
month                                                         
01     165.487125  40466   23.049745  40466  188.536870  40466
02     168.984964  37111   25.549163  37111  194.534127  37111
03     126.101200  40761   22.099114  40761  148.200314  40761
04     151.722805  37378   25.495265  37378  177.218069  37378
05     133.163774  39695   24.974405  39695  158.138179  39695
06     121.357529  41303   22.615137  41303  143.972665  41303
07     143.167743  43680   25.951809  43680  169.119551  43680
08     177.883799  41213   29.496809  41213  207.380608  41213
09     205.389246  33326   31.558393  33326  236.947638  33326
10     279.059524  36439   35.309449  36439  314.368973  36439
11     356.246586  40574   44.942648  40574  401.189234  40574
12     328.621973  18086   36.790335  18086  365.412308  18086

In [12]:
df.reset_index().to_json('data/mecab.json',orient='records')

In [13]:
%%time
from collections import Counter

## df 방식과 counter 방식의 현저한 속도차
#result = df['title_noun'].head(20000).apply(lambda x: pd.value_counts(x.split(" "))).sum(axis = 0)
#result.sort_values(axis=0, ascending=False)
r1 = df['title_me_noun'].tolist()
result = Counter(" ".join(r1).split(" "))

CPU times: user 856 ms, sys: 48 ms, total: 904 ms
Wall time: 904 ms


In [14]:
result.most_common(30)

[('대통령', 17084),
 ('종합', 14622),
 ('한국', 11169),
 ('뉴스', 9603),
 ('국민', 7191),
 ('정부', 7084),
 ('서울', 7016),
 ('중국', 6703),
 ('최순실', 6511),
 ('논란', 6330),
 ('영상', 6239),
 ('경제', 5965),
 ('단독', 5901),
 ('세계', 5865),
 ('기업', 5811),
 ('민주', 5769),
 ('트럼프', 5689),
 ('시장', 5414),
 ('경찰', 5295),
 ('검찰', 5151),
 ('사드', 5128),
 ('여성', 5085),
 ('수사', 4754),
 ('가능', 4752),
 ('조사', 4716),
 ('카드', 4709),
 ('의혹', 4682),
 ('의원', 4578),
 ('사망', 4527),
 ('대표', 4525)]

In [15]:
%%time
from konlpy.tag import Twitter
twitter = Twitter()
def words(sentence):
    word_list = twitter.nouns(sentence)
    return " ".join(word_list)

df['title_noun'] = df['title'].apply(words)
#df.reset_index().to_json('data/twitter.json',orient='records')

CPU times: user 3min 58s, sys: 592 ms, total: 3min 59s
Wall time: 3min 54s
